# Import WOS & Scopus to Infoscience

In [1]:
import sys
import os

sys.path.append(os.path.abspath(".."))

In [2]:
from data_pipeline.main import main
from datetime import datetime
import shutil
from data_pipeline.loader import Loader
from email.mime.multipart import MIMEMultipart
from email.mime.base import MIMEBase
from email import encoders
from email.mime.text import MIMEText

In [3]:
# Création du dossier avec la date actuelle
current_datetime = datetime.now().strftime("%Y_%m_%d-%H_%M_%S")
folder_path = "harvested-data"
os.makedirs(folder_path, exist_ok=True)
path = os.path.join(folder_path, current_datetime)

if not os.path.exists(path):
    os.mkdir(path)

In [4]:
# Exécution du workflow
start = "2025-01-25"
end = "2025-02-03"
df_metadata, df_authors, df_epfl_authors, df_unloaded, df_loaded = main(
    start_date=start, end_date=end
)

2025-02-03 11:51:19,363 - /Users/sicot/Devs/dspace_dev/infoscience-imports/logs/harvesting.log - INFO - Harvesting publications from WOS...


ClientError: 400 Error: Bad Request for url: https://api.clarivate.com/api/wos/?usrQuery=OG%3D%28Ecole+Polytechnique+Federale+de+Lausanne%29&createdTimeSpan=2025-01-25%2B2024-02-03&databaseId=WOS&viewField=UID&count=1&firstRecord=1

In [ ]:
# Sauvegarde des données au format CSV
df_metadata.to_csv(
    os.path.join(path, "ResearchOutput.csv"), index=False, encoding="utf-8"
)
df_authors.to_csv(
    os.path.join(path, "AddressesAndNames.csv"), index=False, encoding="utf-8"
)
df_epfl_authors.to_csv(
    os.path.join(path, "EpflAuthors.csv"), index=False, encoding="utf-8"
)
df_unloaded.to_csv(
    os.path.join(path, "UnloadedDuplicatedPublications.csv"),
    index=False,
    encoding="utf-8",
)
df_loaded.to_csv(
    os.path.join(path, "ImportedPublications.csv"),
    index=False,
    encoding="utf-8",
)